In [4]:
# ### Notebook to parse and organize of the MHSAA Baseball scores for the 2023 season

# # Dependencies
# import pandas as pd
# import requests
# import json

# # global variables

# # Base of the url for the request
# base_url = "https://my.mhsaa.com/DesktopModules/MHSAA-Endpoint/handlers/Scoreboard.ashx?method=teamscoreboard&SportTeamId=2713" # Insert team id here

# # id is going to be a three digit number between 000 and 999, not every number is a valid team id
# id = "000" # Insert team id here

# # end of the url string
# url_end = "&count=50"

# full_url = base_url + team_id + url_end



In [162]:
## How many of the ids are valid vs invalid
# This block is looking at the 2022 season and looking for team ids that resturn usuful data

# # create urls for all the valid ids and save as a list
# base_22 = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID='
# url_end = '&sportTypeCode=BA&gender=M&level=V&year=2021&userid=-1'



HTTPError: HTTP Error 403: Forbidden

In [2]:
import requests
import json
import pandas as pd

##  FUNCTION TO PARSE THE GAME DATA
import re

# Function to parse game data
def parse_game_data(data, game):
    # Extract the team and league information
    team_name = data["Record"]["TeamName"]
    team_id = data["Record"]["SchoolSportTeamId"]
    league_name = data["League"]["Name"]

    # Extract the opponentId and opponentName
    opponent_id = game["Opponents"][0]["SportTeamId"] if game["Opponents"] else None
    opponent_name = game["Opponents"][0]["PopularName"] if game["Opponents"] else None

    # Extract the gameDate and gameTime
    game_date = game["StartDate"]
    game_time = game["TimeText"]

    # Extract homeOrAway
    home_or_away = game["HomeAwayCode"]

    # Extract location info
    location = game["ContestLocationLink"]

    # Extract teamScore, opponentScore, and notes
    team_score = None
    opponent_score = None
    notes = None
    score_text = game["ScoreText"]
    if score_text:
        # Use regular expression to find scores and notes
        match = re.match(r"(\d+)-(\d+)(.*)", score_text)
        if match:
            team_score, opponent_score, notes = match.groups()
            # Convert scores to integers
            team_score = int(team_score)
            opponent_score = int(opponent_score)
            # Trim whitespace from notes
            notes = notes.strip()

    return {
        "teamName": team_name,
        "teamId": team_id,
        "leagueName": league_name,
        "opponentName": opponent_name,
        "opponentId": opponent_id,
        "gameDate": game_date,
        "gameTime": game_time,
        "homeOrAway": home_or_away,
        "location": location,
        "teamScore": team_score,
        "opponentScore": opponent_score,
        "notes": notes
    }



In [3]:


import requests
import json
import pandas as pd

def initialize_parsing(base_url, team_id_start, team_id_end, url_end):
    # Initialize an empty list to hold the parsed data
    parsed_data = []

    # Loop over the range of team IDs
    for i in range(team_id_start, team_id_end+1):
        # Convert the id to a string
        id_str = str(i)
        # Build the URL
        full_url = base_url + id_str + url_end
        # Make the request
        response = requests.get(full_url)

        # Only proceed if the response status code is 200 (HTTP OK)
        if response.status_code == 200:
            # Convert the response to json
            data = response.json()

            # Check to see if the response is valid. If it is, parse the game data
            if 'Contests' in data and data['Contests']:  # Check if 'Contests' key exists and its value is not empty
                for game in data['Contests']:
                    parsed_data.append(parse_game_data(data, game))

    return parsed_data





              teamName   teamId               leagueName  \
0             Hastings  2601492                            
1             Hastings  2601492                            
2             Hastings  2601492                            
3             Hastings  2601492                            
4             Hastings  2601492                            
...                ...      ...                      ...   
16396  South Lyon East  2601982  Lakes Valley Conference   
16397  South Lyon East  2601982  Lakes Valley Conference   
16398  South Lyon East  2601982  Lakes Valley Conference   
16399  South Lyon East  2601982  Lakes Valley Conference   
16400  South Lyon East  2601982  Lakes Valley Conference   

                   opponentName  opponentId             gameDate  gameTime  \
0          Lake Odessa Lakewood   2601838.0  2022-03-28T16:00:00   4:00 PM   
1                     Plainwell   2601912.0  2022-04-12T16:00:00   4:00 PM   
2        Battle Creek Pennfield   2601655.0  

In [ ]:
# # Define the base URL and URL end
# base_url = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID='
# url_end = '&sportTypeCode=BA&gender=M&level=V&year=2021&userid=-1'

# # Define the range of team IDs
# team_id_start = 4000
# team_id_end = 4999

# # Run the function
# parsed_data = initialize_parsing(base_url, team_id_start, team_id_end, url_end)

# # Convert the list of dictionaries to a DataFrame
# df = pd.DataFrame(parsed_data)

# # Print the DataFrame
# print(df)

In [5]:
## Save the data to a csv file
df.to_csv('2022_data_game_level.csv', index=False)

In [181]:
### 2021 Season Data seems to be in the same format as the 2022 season data

# Define the base URL and URL end

# Raw Link: https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID=3969

base_url_21 = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID='

end_21 = '&sportTypeCode=BA&gender=M&level=V&year=2020&userid=-1'

id21_start = 0000
id21_end = 9999

parsed_data_21 = initialize_parsing(base_url_21, id21_start, id21_end, end_21)

df_21 = pd.DataFrame(parsed_data_21)

print(df_21)

SSLError: HTTPSConnectionPool(host='my.mhsaa.com', port=443): Max retries exceeded with url: /DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID=9224&sportTypeCode=BA&gender=M&level=V&year=2020&userid=-1 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1129)')))

In [176]:
## Save the data to a csv file with date stamp

import datetime

now = datetime.datetime.now()

date_stamp = now.strftime("%Y-%m-%d")

df_21.to_csv('2021_data_game_level_' + date_stamp + '.csv', index=False)

In [180]:
# Inspect data for 2021 season

df_21.info()

# how many unique teams are there in the data set

df_21['teamName'].nunique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3694 entries, 0 to 3693
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   teamName       3694 non-null   object 
 1   teamId         3694 non-null   int64  
 2   leagueName     3694 non-null   object 
 3   opponentName   3530 non-null   object 
 4   opponentId     3530 non-null   float64
 5   gameDate       3694 non-null   object 
 6   gameTime       3694 non-null   object 
 7   homeOrAway     3694 non-null   object 
 8   location       3694 non-null   object 
 9   teamScore      1495 non-null   float64
 10  opponentScore  1495 non-null   float64
 11  notes          1495 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 346.4+ KB


124

In [4]:
# 2020 Season Cancelled due to COVID-19



# 2019 Season Data seems to be in the same format as the 2022 season data

## Raw URL
raw_19 = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID=3969&sportTypeCode=BA&gender=M&level=V&year=2018&userid=-1'

base_url_19 = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID='

end_19 = '&sportTypeCode=BA&gender=M&level=V&year=2018&userid=-1'

id19_start = 3000
id19_end = 3999

parsed_data_19 = initialize_parsing(base_url_19, id19_start, id19_end, end_19)

df_19 = pd.DataFrame(parsed_data_19)

print(df_19)


                     teamName   teamId  \
0                    Brighton  2163805   
1                    Brighton  2163805   
2                    Brighton  2163805   
3                    Brighton  2163805   
4                    Brighton  2163805   
...                       ...      ...   
3771  Taylor Trillium Academy  2163898   
3772  Taylor Trillium Academy  2163898   
3773  Taylor Trillium Academy  2163898   
3774  Taylor Trillium Academy  2163898   
3775  Taylor Trillium Academy  2163898   

                                        leagueName  \
0     Kensington Lakes Activities Association-Gold   
1     Kensington Lakes Activities Association-Gold   
2     Kensington Lakes Activities Association-Gold   
3     Kensington Lakes Activities Association-Gold   
4     Kensington Lakes Activities Association-Gold   
...                                            ...   
3771      Michigan Metro Athletic Conference-Black   
3772      Michigan Metro Athletic Conference-Black   
3773     

In [ ]:
df_19.to_csv('2019_data_game_level_' + date_stamp + '.csv', index=False)

In [182]:
# 2018 Season Data

# Raw URL

raw_18 = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID=3969&sportTypeCode=BA&gender=M&level=V&year=2017&userid=-1'

base_url_18 = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID='

end_18 = '&sportTypeCode=BA&gender=M&level=V&year=2017&userid=-1'

id18_start = 1000
id18_end = 1050

parsed_data_18 = initialize_parsing(base_url_18, id18_start, id18_end, end_18)

df_18 = pd.DataFrame(parsed_data_18)

print(df_18)




Empty DataFrame
Columns: []
Index: []


In [ ]:
#2017 Season Data

# Raw Data

raw_17 = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID=3969&sportTypeCode=BA&gender=M&level=V&year=2016&userid=-1'

base_url_17 = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID='

end_17 = '&sportTypeCode=BA&gender=M&level=V&year=2016&userid=-1'

id17_start = 1
id17_end = 999

parsed_data_17 = initialize_parsing(base_url_17, id17_start, id17_end, end_17)

df_17 = pd.DataFrame(parsed_data_17)

print(df_17)


In [ ]:
#2016 Season Data

# RAW Data

raw_16 = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID=3969&sportTypeCode=BA&gender=M&level=V&year=2015&userid=-1'

base_url_16 = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID='

end_16 = '&sportTypeCode=BA&gender=M&level=V&year=2015&userid=-1'

id16_start = 1
id16_end = 999

parsed_data_16 = initialize_parsing(base_url_16, id16_start, id16_end, end_16)

df_16 = pd.DataFrame(parsed_data_16)

print(df_16)


In [154]:
## Take a look at the data
# print(df.head())

# print(df.tail())

# df.info()

# ## Only show the games with a score
# print(df[df["teamScore"].notnull()])
# print(df[df["opponentScore"].notnull()])


# Show the values that are in the notes column
# print(df["notes"].value_counts())

                   5458
(5 Inn)             883
(6 Inn)             553
(3 Inn)             299
(4 Inn)             269
(8 Inn)             105
Forfeit              55
(9 Inn)              51
(1 Inn)              13
(2 Inn)               8
(10 Inn)              6
(11 Inn)              4
(12 Inn)              4
(6 Inn) Forfeit       3
(13 Inn)              2
(1 Inn) Forfeit       1
Name: notes, dtype: int64


In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16401 entries, 0 to 16400
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   teamId         16401 non-null  int64  
 1   opponentId     15830 non-null  float64
 2   gameDate       16401 non-null  object 
 3   homeOrAway     16401 non-null  object 
 4   teamScore      7714 non-null   float64
 5   opponentScore  7714 non-null   float64
 6   notes          7714 non-null   object 
dtypes: float64(3), int64(1), object(3)
memory usage: 897.1+ KB


In [167]:
## Clean up the 2022 data and save temp copy to review

## look at a sample of rows that don't have a score
# print(df[df["teamScore"].isnull()].sample(10))

# ## look at a sample of rows that do have scores
# print(df[df["teamScore"].notnull()].sample(10))

# print first 20 rows
print(df.head(20))

# drop rows where teamScore or opponentScore is null

## Save the data to a csv file
df.to_csv('2022_data_raw_TEMP.csv')



     teamId  opponentId             gameDate homeOrAway  teamScore  \
0   2601492   2601838.0  2022-03-28T16:00:00          A        NaN   
1   2601492   2601912.0  2022-04-12T16:00:00          H        1.0   
2   2601492   2601655.0  2022-04-14T16:00:00          A        NaN   
3   2601492         NaN  2022-04-16T09:00:00          H        NaN   
4   2601492         NaN  2022-04-16T13:00:00          H        NaN   
5   2601492   2601699.0  2022-04-26T16:00:00          H        NaN   
6   2601492   2601482.0  2022-04-28T16:00:00          A        NaN   
7   2601492   2601781.0  2022-05-05T16:15:00          A        NaN   
8   2601492   2601781.0  2022-05-05T18:30:00          A        NaN   
9   2601492   2601454.0  2022-05-07T09:00:00          A        NaN   
10  2601492   2601926.0  2022-05-10T16:00:00          H        5.0   
11  2601492   2601926.0  2022-05-10T18:00:00          H        NaN   
12  2601492   2601917.0  2022-05-11T16:00:00          A        NaN   
13  2601492   260157

In [142]:
## NOTES

##  IT WOULD BE INTERESTING TO SEE IF THE DISTANCE FROM THE SCHOOL TO THE GAME HAS ANY EFFECT ON THE SCORE

## NEXT STEPS TO TAKE
# NEED TO LINK TO THE SCHOOL DATA TO GET THE SCHOOL NAME AND ADDRESS ALONG WITH ALL THE OTHER STUFF I HAVE ALREADY COLLECTED

# - MERGE GEOGRAPHIC DATA INTO THE DATAFRAME. SCHOOL STREET ADDRESS CITY STATE SHOULD BE PRETTY EASY TO GET.
# MHSAA.COM 

# OTHER INTERESTING VIEWS. WITH GEO LOCATION FOR EACH WE COULD LOOK AT HEAT MAPS OF WINS



In [143]:
df.head()


NameError: name 'df' is not defined

In [141]:
## Show the output of id 4998

## Check to see if other years are available
import requests
import json

## 2022 season
valid_ids = []
base_22 = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID='
url_end = '&sportTypeCode=BA&gender=M&level=V&year=2021&userid=-1'

# Convert the id to a string
id = str(4020).zfill(4)
# Build the url
full_url = base_22 + id + url_end
# Make the request
response = requests.get(full_url)

# Convert the response to json
data = response.json()

# Print the json
print(json.dumps(data, indent=4, sort_keys=True))

{
    "Contests": [
        {
            "AlternativeLocation": null,
            "ArbiterKey": 71080568,
            "BroadcastInfo": "<img src='/DesktopModules/MHSAA-Schedules/images/broadcast.png' class='mbIcon' id='MB3763620' />",
            "CanEdit": false,
            "ChildContests": [],
            "ContestLocation": "",
            "ContestLocationLink": "http://maps.google.com/maps?q=Kalamazoo+Christian+High+School+2121+Stadium+Dr++Kalamazoo%2c+MI+49008",
            "ContestName": null,
            "ContestTieInfo": null,
            "ContestType": 1,
            "DateText": "3/23/2022",
            "EditInfo": null,
            "GameStatusType": "CAN",
            "HasArbiterInfo": true,
            "HasBroadcastInfo": false,
            "HasMHSAATV": false,
            "HasNotes": false,
            "HomeAwayCode": "H",
            "InDispute": false,
            "IsLeagueGame": true,
            "IsLocked": false,
            "LockInfo": "",
            "MHSAATVEventId

In [134]:
## Check if id all id numbers return a valid response
# select 1000 random numbers between 0000 and 9999
import random
import requests
import json

# Create a list of 1000 random numbers
random_ids = random.sample(range(0, 9999), 1000)

# Convert the list of random numbers to a list of strings
random_ids = [str(i).zfill(4) for i in random_ids]

# Create a list to hold the valid ids
valid_ids = []

# Loop through the random ids
for id in random_ids:
    # Build the url
    full_url = base_22 + id + url_end
    # Make the request
    response = requests.get(full_url)
    # Convert the response to json
    data = response.json()
    # Check to see if the response is valid. If it is add it to the list of valid ids
    if len(data) > 0:  # check if list is not empty
        valid_ids.append(id)
        print(f"Valid id: {id}")
    else:
        print(f"Invalid id: {id}")
        

Valid id: 0821
Valid id: 6343
Valid id: 2543
Valid id: 5383
Valid id: 1480
Valid id: 5846
Valid id: 7021
Valid id: 9878
Valid id: 4346
Valid id: 1985
Valid id: 5426
Valid id: 8464
Valid id: 9833
Valid id: 3047
Valid id: 1749
Valid id: 9695
Valid id: 3185
Valid id: 8949
Valid id: 5954
Valid id: 9789
Valid id: 9618
Valid id: 1633
Valid id: 2884
Valid id: 6358
Valid id: 4457
Valid id: 4737
Valid id: 8358
Valid id: 4197
Valid id: 9815
Valid id: 1565
Valid id: 2993
Valid id: 3959
Valid id: 5893
Valid id: 7424
Valid id: 0617
Valid id: 1049
Valid id: 1911
Valid id: 0964
Valid id: 7447
Valid id: 5278
Valid id: 7757
Valid id: 6479
Valid id: 0958
Valid id: 9483
Valid id: 1708
Valid id: 4297
Valid id: 8905
Valid id: 1429
Valid id: 1451
Valid id: 1558
Valid id: 9386
Valid id: 6356
Valid id: 2587
Valid id: 9916
Valid id: 6894
Valid id: 2414
Valid id: 1642
Valid id: 5133
Valid id: 6041
Valid id: 8397
Valid id: 6984
Valid id: 8545
Valid id: 2497
Valid id: 8996
Valid id: 8616
Valid id: 1101
Valid id: 

KeyboardInterrupt: 

In [5]:
# Dependencies
import pandas as pd
import requests
import json
import time
from tqdm import tqdm

# Base of the url for the request
base_url = "https://my.mhsaa.com/DesktopModules/MHSAA-Endpoint/handlers/Scoreboard.ashx?method=teamscoreboard&SportTeamId=2713" # Insert team id here

# end of the url string
url_end = "&count=50"

# Main dataframe to hold all scores
df_all_scores = pd.DataFrame()

# Counter for valid team IDs
valid_ids = 0

# Start the timer
start_time = time.time()

# Loop over all possible team IDs
for i in tqdm(range(0, 1000), desc="Processing team IDs"):
    # Construct the team ID as a three-digit string
    team_id = str(i).zfill(3)

    # Construct the full URL
    full_url = base_url + team_id + url_end

    # Make the request
    response = requests.get(full_url)

    # If the request was successful
    if response.status_code == 200:
        # Load the JSON data
        data = json.loads(response.text)

        # Convert the data to a DataFrame
        df_scores = pd.DataFrame(data)

        # If the DataFrame is not empty
        if not df_scores.empty:
            # Increment the counter
            valid_ids += 1
            # Combine 'homeschool' and 'awayschool' columns into one Series
            teams = pd.concat([df_scores['homeschool'], df_scores['awayschool']], ignore_index=True)

            # Determine the primary team based on frequency
            primary_team = teams.value_counts().idxmax()

            # Initialize new columns
            df_scores['primary_team'] = primary_team
            df_scores['opponent'] = ''
            df_scores['home/away'] = ''
            df_scores['primary_score'] = 0.0
            df_scores['opp_score'] = 0.0

            # Fill in the new columns based on the 'awayschool' and 'homeschool' columns
            for i, row in df_scores.iterrows():
                if row['awayschool'] == primary_team:
                    df_scores.at[i, 'opponent'] = row['homeschool']
                    df_scores.at[i, 'home/away'] = 'away'
                    df_scores.at[i, 'primary_score'] = row['awayscore']
                    df_scores.at[i, 'opp_score'] = row['homescore']
                else:
                    df_scores.at[i, 'opponent'] = row['awayschool']
                    df_scores.at[i, 'home/away'] = 'home'
                    df_scores.at[i, 'primary_score'] = row['homescore']
                    df_scores.at[i, 'opp_score'] = row['awayscore']

            # Classify each game as a win, loss or tie for the primary team
            df_scores['result'] = ''
            for i, row in df_scores.iterrows():
                if row['primary_score'] > row['opp_score']:
                    df_scores.at[i, 'result'] = 'win'
                elif row['primary_score'] < row['opp_score']:
                    df_scores.at[i, 'result'] = 'loss'
                else:
                    df_scores.at[i, 'result'] = 'tie'

            # Drop the original 'homeschool', 'awayschool', 'homescore', 'awayscore' columns
            df_scores = df_scores.drop(columns=['homeschool', 'awayschool', 'homescore', 'awayscore',
                                                'homeclass', 'awayclass'])

            # Rename the 'scoredate' and 'scoretime' columns to match your desired column names
            df_scores = df_scores.rename(columns={'scoredate': 'date', 
                                                'scoretime': 'time', 
                                                'additionalscoreinfo': 'notes',
                                                'leveltypecode': 'level_code',})

            # Reorder the columns
            df_scores = df_scores[['date', 'time', 'primary_team', 'opponent', 'home/away', 'primary_score', 
                                'opp_score', 'notes', 'level_code']]
            # Append the reorganized DataFrame to the main DataFrame
            df_all_scores = df_all_scores.append(df_scores, ignore_index=True)

    # Delay for one second
    # time.sleep(1)


# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the statistics
print(f"Execution time: {elapsed_time} seconds")
print(f"Valid team IDs found: {valid_ids}")
print(f"Total games represented: {df_all_scores.shape[0] // 2}")




Processing team IDs:   0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_17320\1836002613.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_scores = df_all_scores.append(df_scores, ignore_index=True)
Processing team IDs:   0%|          | 1/1000 [00:00<13:01,  1.28it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_17320\1836002613.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_scores = df_all_scores.append(df_scores, ignore_index=True)
Processing team IDs:   0%|          | 2/1000 [00:01<07:50,  2.12it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_17320\1836002613.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_scores = df_all_scores.append(df_scores, ignore_

Execution time: 225.83016324043274 seconds
Valid team IDs found: 608
Total games represented: 7780


In [6]:
df_all_scores.info()

df_all_scores.head(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15561 entries, 0 to 15560
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           15561 non-null  object 
 1   time           15561 non-null  object 
 2   primary_team   15561 non-null  object 
 3   opponent       15561 non-null  object 
 4   home/away      15561 non-null  object 
 5   primary_score  15561 non-null  float64
 6   opp_score      15561 non-null  float64
 7   notes          15561 non-null  object 
 8   level_code     15561 non-null  object 
dtypes: float64(2), object(7)
memory usage: 1.1+ MB


,date,time,primary_team,opponent,home/away,primary_score,opp_score,notes,level_code
0,04/07/2023,4:30PM,Detroit Communication Media Arts,Detroit Southeastern,home,17.0,2.0,(3 Inn),V
1,04/05/2023,4:00PM,Detroit Communication Media Arts,Detroit Southeastern,away,17.0,2.0,(3 Inn),V
2,06/03/2023,10:00AM,Harbor Springs Harbor Light Christian,Harbor Springs,home,3.0,6.0,,V
3,05/30/2023,5:00PM,Harbor Springs Harbor Light Christian,Burt Lake Northern Michigan Christian,home,12.0,2.0,(5 Inn),V
4,05/15/2023,5:30PM,Harbor Springs Harbor Light Christian,Mackinaw City,home,3.0,5.0,,V
5,05/15/2023,4:00PM,Harbor Springs Harbor Light Christian,Mackinaw City,home,10.0,8.0,,V
6,05/11/2023,5:30PM,Harbor Springs Harbor Light Christian,Onaway,home,6.0,5.0,,V
7,05/11/2023,4:00PM,Harbor Springs Harbor Light Christian,Onaway,home,3.0,6.0,,V
8,05/09/2023,5:45PM,Harbor Springs Harbor Light Christian,Posen,home,8.0,11.0,,V
9,05/09/2023,4:00PM,Harbor Springs Harbor Light Christian,Posen,home,15.0,0.0,,V


In [10]:
import pandas as pd
## Save the DataFrame to a CSV file with the date and time it was created

# Get the current date and time
now = pd.Timestamp.now()

# Format the date and time
now = now.strftime("%Y-%m-%d_%H-%M")

# Save the DataFrame to a CSV file
df_all_scores.to_csv('MHSAA_all_games_Thu_Jul_22_2021_11-30.csv', index=False)

In [109]:
## Reload the data into a clean Dataframe so I don't need to continually run the above code

# Load the CSV file into a DataFrame
df_all_scores = pd.read_csv('MHSAA_all_games_Thu_Jul_22_2021_11-30.csv')

team_game_counts = df_all_scores['primary_team'].value_counts()
df_all_scores['games_played'] = df_all_scores['primary_team'].map(team_game_counts)

In [110]:
## Calculate the teams stats home vs away




# Create a column for the result of each game (from the proimary team's perspective)
df_all_scores['result'] = 'tie'
for i, row in df_all_scores.iterrows():
    if row['home/away'] == 'home':
        if row['primary_score'] > row['opp_score']:
            df_all_scores.at[i, 'result'] = 'win'
        elif row['primary_score'] < row['opp_score']:
            df_all_scores.at[i, 'result'] = 'loss'
        else:
            df_all_scores.at[i, 'result'] = 'tie'
    else:
        if row['primary_score'] > row['opp_score']:
            df_all_scores.at[i, 'result'] = 'win'
        elif row['primary_score'] < row['opp_score']:
            df_all_scores.at[i, 'result'] = 'loss'
        else:
            df_all_scores.at[i, 'result'] = 'tie'

# Calculate the overall avg runs scored and allowed
df_all_scores['overall_avg_runs_scored'] = df_all_scores['primary_score'].mean()
df_all_scores['overall_avg_runs_allowed'] = df_all_scores['opp_score'].mean()
# Calculate the differential
df_all_scores['overall_avg_run_differential'] = df_all_scores['overall_avg_runs_scored'] - df_all_scores['overall_avg_runs_allowed']



## Create a DataFrame with the home games
df_home_games = df_all_scores[df_all_scores['home/away'] == 'home']
# Calculate the number of home games played
df_home_games['home_games'] = df_home_games['primary_team'].value_counts()
# Calculate each team's record at home
df_home_games = df_home_games.assign(home_wins = df_home_games['result'].eq('win'))
df_home_games = df_home_games.assign(home_losses = df_home_games['result'].eq('loss'))
df_home_games = df_home_games.assign(home_ties = df_home_games['result'].eq('tie'))

df_home_games_stats = df_home_games.groupby('primary_team').agg({
    'home_wins': 'sum',
    'home_losses': 'sum',
    'home_ties': 'sum',
    'primary_score': 'sum',
    'opp_score': 'sum',
})

df_home_games_stats['home_games'] = df_home_games_stats[['home_wins', 'home_losses', 'home_ties']].sum(axis=1)
df_home_games_stats['home_total_runs_scored'] = df_home_games_stats['primary_score'].sum()
df_home_games_stats['home_total_runs_allowed'] = df_home_games_stats['opp_score'].sum()
df_home_games_stats['home_avg_runs_scored'] = df_home_games_stats['primary_score'] / df_home_games_stats['home_games']
df_home_games_stats['home_avg_runs_allowed'] = df_home_games_stats['opp_score'] / df_home_games_stats['home_games']
df_home_games_stats['home_run_diff'] = df_home_games_stats['home_avg_runs_scored'] - df_home_games_stats['home_avg_runs_allowed']
df_home_games_stats['home_win_pct'] = df_home_games_stats['home_wins'] / df_home_games_stats['home_games']

# Repeat the same process for away games
df_away_games = df_all_scores[df_all_scores['home/away'] == 'away']

# Calculate the number of away games played
df_away_games['away_games'] = df_away_games['primary_team'].value_counts()
# Calculate each team's record away
df_away_games = df_away_games.assign(away_wins = df_away_games['result'].eq('win'))
df_away_games = df_away_games.assign(away_losses = df_away_games['result'].eq('loss'))
df_away_games = df_away_games.assign(away_ties = df_away_games['result'].eq('tie'))

df_away_games_stats = df_away_games.groupby('primary_team').agg({
    'away_wins': 'sum',
    'away_losses': 'sum',
    'away_ties': 'sum',
    'primary_score': 'sum',
    'opp_score': 'sum',
})

df_away_games_stats['away_games'] = df_away_games_stats[['away_wins', 'away_losses', 'away_ties']].sum(axis=1)
df_away_games_stats['away_total_runs_scored'] = df_away_games_stats['primary_score'].sum()
df_away_games_stats['away_total_runs_allowed'] = df_away_games_stats['opp_score'].sum()
df_away_games_stats['away_avg_runs_scored'] = df_away_games_stats['primary_score'] / df_away_games_stats['away_games']
df_away_games_stats['away_avg_runs_allowed'] = df_away_games_stats['opp_score'] / df_away_games_stats['away_games']
df_away_games_stats['away_run_diff'] = df_away_games_stats['away_avg_runs_scored'] - df_away_games_stats['away_avg_runs_allowed']
df_away_games_stats['away_win_pct'] = df_away_games_stats['away_wins'] / df_away_games_stats['away_games']




# Recombine the home and away DataFrames by merging on the team name
df_team_stats = pd.merge(df_home_games_stats, df_away_games_stats, how='outer', on='primary_team')

# Recreate the overall record
df_team_stats['wins'] = df_team_stats['home_wins'] + df_team_stats['away_wins']
df_team_stats['losses'] = df_team_stats['home_losses'] + df_team_stats['away_losses']
df_team_stats['ties'] = df_team_stats['home_ties'] + df_team_stats['away_ties']
df_team_stats['games'] = df_team_stats['wins'] + df_team_stats['losses'] + df_team_stats['ties']
df_team_stats['overall_win_pct'] = df_team_stats['wins'] / df_team_stats['games']

# Caluclate find the difference between home and away stats
df_team_stats['win_pct_change'] = df_team_stats['home_win_pct'] - df_team_stats['away_win_pct']
df_team_stats['runs_scored_change'] = df_team_stats['home_avg_runs_scored'] - df_team_stats['away_avg_runs_scored']
df_team_stats['runs_allowed_change'] = df_team_stats['home_avg_runs_allowed'] - df_team_stats['away_avg_runs_allowed']
df_team_stats['run_diff_change'] = df_team_stats['home_run_diff'] - df_team_stats['away_run_diff']









C:\Users\Justin\AppData\Local\Temp\ipykernel_17320\4193056644.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_home_games['home_games'] = df_home_games['primary_team'].value_counts()
C:\Users\Justin\AppData\Local\Temp\ipykernel_17320\4193056644.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_away_games['away_games'] = df_away_games['primary_team'].value_counts()


In [111]:
# df_team_stats.head(20)

In [112]:
## Reorder the columns
df_team_stats = df_team_stats[['games', 'wins', 'losses', 'ties', 'overall_win_pct', 
                               'home_games', 'home_wins', 'home_losses', 'home_ties', 'home_win_pct', 
                               'home_avg_runs_scored', 'home_avg_runs_allowed', 'home_run_diff', 
                               'away_games', 'away_wins', 'away_losses', 'away_ties', 'away_win_pct', 
                               'away_avg_runs_scored', 'away_avg_runs_allowed', 'away_run_diff', 'win_pct_change',
                                'runs_scored_change', 'runs_allowed_change', 'run_diff_change']]

## Try a new order of columns

df_team_stats_2 = df_team_stats[['games','home_games','away_games',
                                 'wins','home_wins','away_wins',
                                 'losses','home_losses','away_losses',
                                 'ties','home_ties','away_ties',
                                 'overall_win_pct','home_win_pct','away_win_pct',
                                 'home_avg_runs_scored','away_avg_runs_scored','runs_scored_change',
                                 'home_avg_runs_allowed','away_avg_runs_allowed','runs_allowed_change',
                                 'home_run_diff','away_run_diff','run_diff_change']]
# # drop unneeded columns
# df_team_stats = df_team_stats.drop(['primary_score_x', 'opp_score_x', 'primary_score_y', 'opp_score_y'], axis=1)

In [113]:
## Save as a CSV file for manual review
df_team_stats_2.to_csv('MHSAA_team_stats_Thu_Jul_22_2021_11-30.csv')

In [17]:
## Calculate the run differential for primary team

## Grouby primary team and sume the runs scored and runs allowed
df_team_runs = df_all_scores.groupby('primary_team').sum()

## Create a column for tyhe number of games the primary team played
df_team_runs['games_played'] = df_all_scores['primary_team'].value_counts()

## Calculate the run differential
df_team_runs['run_diff'] = df_team_runs['primary_score'] - df_team_runs['opp_score']

## Sort the DataFrame by run differential
df_team_runs = df_team_runs.sort_values('run_diff', ascending=False)

## Calculate the average run differential
df_team_runs['avg_run_diff'] = df_team_runs['run_diff'] / df_team_runs['games_played']

## Print the top 10 teams by run differential
# df_team_runs.head(10)

,primary_score,opp_score,games_played,run_diff,avg_run_diff
primary_team,,,,,
Decatur,409.0,77.0,31,332.0,10.709677
Maple City Glen Lake,391.0,81.0,38,310.0,8.157895
Zeeland East,406.0,108.0,41,298.0,7.268293
Olivet,357.0,77.0,41,280.0,6.829268
Beal City,358.0,99.0,38,259.0,6.815789
Muskegon Catholic Central,372.0,113.0,39,259.0,6.641026
Mount Pleasant Sacred Heart,376.0,130.0,39,246.0,6.307692
Bay City John Glenn,343.0,98.0,42,245.0,5.833333
Algonac,314.0,71.0,40,243.0,6.075000


In [18]:
## SHow the teams with the lowest run differential
# df_team_runs.tail(20)

,primary_score,opp_score,games_played,run_diff,avg_run_diff
primary_team,,,,,
Waldron,5.0,229.0,13,-224.0,-17.230769
Greenville,117.0,341.0,39,-224.0,-5.743590
St Louis,98.0,327.0,31,-229.0,-7.387097
Westland Huron Valley Lutheran,51.0,280.0,15,-229.0,-15.266667
Athens,26.0,258.0,17,-232.0,-13.647059
Battle Creek Central,34.0,275.0,18,-241.0,-13.388889
Bloomingdale,27.0,268.0,16,-241.0,-15.062500
Saginaw Arthur Hill,27.0,273.0,15,-246.0,-16.400000
Bangor,79.0,328.0,23,-249.0,-10.826087


In [20]:
# ## Soft the teams by average run differential

# df_team_runs = df_team_runs.sort_values('avg_run_diff', ascending=False)

# ## Filter out teams that have less than 10 games played
# filter_df_team_runs = df_team_runs[df_team_runs['games_played'] >= 10]

# ## Print the top 20 teams by average run differential
# filter_df_team_runs.head(20)



,primary_score,opp_score,games_played,run_diff,avg_run_diff
primary_team,,,,,
Decatur,409.0,77.0,31,332.0,10.709677
Marion,232.0,52.0,18,180.0,10.000000
Maple City Glen Lake,391.0,81.0,38,310.0,8.157895
Zeeland East,406.0,108.0,41,298.0,7.268293
Charlevoix,235.0,51.0,26,184.0,7.076923
Indian River Inland Lakes,152.0,32.0,17,120.0,7.058824
Gaylord St Mary,267.0,87.0,26,180.0,6.923077
Merrill,361.0,119.0,35,242.0,6.914286
Dearborn Heights Annapolis,296.0,97.0,29,199.0,6.862069


In [23]:
# ## Calculate the average runs scored and allowed per game then display the top 20 teams for each
# ## Limit results to teams that played 10 or more games

# ## Calculate the average runs scored per game
# filter_df_team_runs['avg_runs_scored'] = filter_df_team_runs['primary_score'] / filter_df_team_runs['games_played']

# ## Calculate the average runs allowed per game
# filter_df_team_runs['avg_runs_allowed'] = filter_df_team_runs['opp_score'] / filter_df_team_runs['games_played']

# ## Sort the DataFrame by average runs scored
# filter_df_team_runs = filter_df_team_runs.sort_values('avg_runs_scored', ascending=False)

# ## Print the top 20 teams by average runs scored
# filter_df_team_runs.head(20)

# ## Sort the DataFrame by average runs allowed
# # filter_df_team_runs = filter_df_team_runs.sort_values('avg_runs_allowed', ascending=True)

# ## Print the top 20 teams by average runs allowed
# filter_df_team_runs.head(20)


,primary_score,opp_score,games_played,run_diff,avg_run_diff,avg_runs_scored,avg_runs_allowed
primary_team,,,,,,,
Decatur,409.0,77.0,31,332.0,10.709677,13.193548,2.483871
Marion,232.0,52.0,18,180.0,10.000000,12.888889,2.888889
Bloomfield Hills Roeper,237.0,102.0,21,135.0,6.428571,11.285714,4.857143
Merrill,361.0,119.0,35,242.0,6.914286,10.314286,3.400000
Maple City Glen Lake,391.0,81.0,38,310.0,8.157895,10.289474,2.131579
Gaylord St Mary,267.0,87.0,26,180.0,6.923077,10.269231,3.346154
West Bloomfield Frankel Jewish Academy,123.0,74.0,12,49.0,4.083333,10.250000,6.166667
Dearborn Heights Annapolis,296.0,97.0,29,199.0,6.862069,10.206897,3.344828
Lawrence,322.0,121.0,32,201.0,6.281250,10.062500,3.781250


In [28]:
# Show the resulting dataframe
df_team_runs.head(20)

,home_wins,home_losses,home_ties,primary_score,opp_score,home_games,avg_runs_scored_home,avg_runs_allowed_home,away_wins,away_losses,away_ties,primary_score,opp_score,away_games
primary_team,,,,,,,,,,,,,,
Ada Forest Hills Eastern,14.0,5.0,0.0,122.0,57.0,19.0,6.421053,3.000000,2.0,18.0,0.0,158.0,58.0,20.0
Addison,7.0,8.0,0.0,93.0,65.0,15.0,6.200000,4.333333,7.0,9.0,0.0,88.0,103.0,16.0
Adrian,12.0,7.0,0.0,123.0,67.0,19.0,6.473684,3.526316,2.0,19.0,0.0,207.0,56.0,21.0
Adrian Lenawee Christian,0.0,4.0,0.0,27.0,75.0,4.0,6.750000,18.750000,7.0,1.0,0.0,33.0,85.0,8.0
Adrian Madison,8.0,7.0,0.0,107.0,88.0,15.0,7.133333,5.866667,6.0,13.0,0.0,193.0,76.0,19.0
Akron-Fairgrove,6.0,3.0,0.0,74.0,52.0,9.0,8.222222,5.777778,10.0,2.0,1.0,33.0,142.0,13.0
Alanson,2.0,14.0,0.0,77.0,195.0,16.0,4.812500,12.187500,2.0,0.0,0.0,1.0,24.0,2.0
Algonac,26.0,2.0,0.0,217.0,48.0,28.0,7.750000,1.714286,1.0,11.0,0.0,97.0,23.0,12.0
Allegan,4.0,10.0,0.0,54.0,94.0,14.0,3.857143,6.714286,3.0,6.0,1.0,57.0,40.0,10.0
